In [1]:
import csv
import datetime
import pandas as pd
import sounddevice as sd
from scipy.io.wavfile import write
import wavio as wv
import librosa
from tensorflow import keras
import numpy as np
import pandas as pd
import os
from datetime import datetime
from datetime import date






In [2]:
model = keras.models.load_model('modelo.p')

CSV_FILE_PATH = "./base/archive/esc50.csv"
DATA_PATH = "./base/"

df = pd.read_csv(CSV_FILE_PATH, sep=',')
df.head()


,filename,target,category
0,1-34094-A-5.wav,5,gato
1,1-34094-B-5.wav,5,gato
2,1-47819-A-5.wav,5,gato
3,1-47819-B-5.wav,5,gato
4,1-47819-C-5.wav,5,gato


In [3]:
#df=df.drop(['fold','esc10','src_file','take'], axis=1)

classes = df['category'].unique()
print("Classes are: ",classes)
print("# of Classes are: ",classes[0])

class_dict = {i:x for x,i in enumerate(classes)}

class_dict

Classes are:  ['gato' 'cachorro' 'batendo_porta' 'silencio']
# of Classes are:  gato


{'gato': 0, 'cachorro': 1, 'batendo_porta': 2, 'silencio': 3}

In [4]:
df = df.drop_duplicates(subset=['filename'])
df['target'] = df['category'].map(class_dict)
df

,filename,target,category
0,1-34094-A-5.wav,0,gato
1,1-34094-B-5.wav,0,gato
2,1-47819-A-5.wav,0,gato
3,1-47819-B-5.wav,0,gato
4,1-47819-C-5.wav,0,gato
...,...,...,...
591,silencio146.wav,3,silencio
592,silencio147.wav,3,silencio
593,silencio148.wav,3,silencio
594,silencio149.wav,3,silencio


In [5]:
def remove_initial_comma(file_path):
    try:
        # Abra o arquivo em modo de leitura
        with open(file_path, 'r') as arquivo:
            # Leia o conteúdo do arquivo
            conteudo = arquivo.read()

        # Remova a vírgula inicial, se existir
        conteudo_sem_virgula_inicial = conteudo[1:]
        

        # Abra o arquivo novamente em modo de escrita
        with open(file_path, 'w') as arquivo:
            # Escreva o conteúdo modificado de volta no arquivo
            arquivo.write(conteudo_sem_virgula_inicial)

        print("Vírgula inicial removida com sucesso.")
    except FileNotFoundError:
        print(f"O arquivo '{file_path}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo: {str(e)}")



In [6]:
    try:
        # Abra o arquivo em modo de leitura
        with open(file_path, 'r') as arquivo:
            # Leia o conteúdo do arquivo
            conteudo = arquivo.read()

        # Remova a vírgula inicial, se existir
        conteudo_sem_virgula_inicial = conteudo[1:]
        
    except FileNotFoundError:
        print(f"O arquivo '{file_path}' não foi encontrado.")
    except Exception as e:
        print(f"Ocorreu um erro ao processar o arquivo: {str(e)}")

Ocorreu um erro ao processar o arquivo: name 'file_path' is not defined


In [ ]:




freq = 44100
duration = 5

#Criação do arquivo para guardar os dados classificados

# Obtenha a data atual
data_atual = date.today()

# Converta a data para uma string no formato AAAA-MM-DD
nome_pasta = data_atual.strftime('%Y-%m-%d')

# Especifique o caminho onde você deseja criar a pasta
caminho_pasta = './assets'  # Substitua pelo caminho desejado

# Crie o caminho completo para a nova pasta
caminho_completo = os.path.join(caminho_pasta, nome_pasta)

# Verifique se a pasta já existe antes de criá-la
if not os.path.exists(caminho_completo):
    os.makedirs(caminho_completo)





data_atual = datetime.now()

data = data_atual.strftime("%Y-%m-%d_%H-%M-%S")

diames= data_atual.strftime("%Y-%m-%d") + ".csv"

horasec = data_atual.strftime("%H:%M:%S.%f")







if os.path.exists(diames) == False:  
 open(diames,'a').close()
 df = pd.DataFrame(columns=['id','data','hora','classicacao'])
 df.to_csv(diames)
 # Exemplo de uso da função

remove_initial_comma(diames)

    
    
try:
 df = pd.read_csv(diames)
 id=0
except Exception as e:
    print(e)

while True:
    data_atual = datetime.now()
    data = data_atual.strftime("%Y-%m-%d_%H-%M-%S")
    diames= data_atual.strftime("%Y-%m-%d") + ".csv"
    horasec = data_atual.strftime("%H:%M:%S.%f")

    recording = sd.rec(int(duration * freq), samplerate=freq, channels=2)
    sd.wait()
    wv.write("./temp/recording1.wav", recording, freq, sampwidth=2)
    wv.write("./temp/recording"+ str(data_atual.second) +".wav", recording, freq, sampwidth=2)
    
    print("\ndone...")

    X = []
    sig, sr = librosa.load("./temp/recording1.wav")
    
    mfcc_ = librosa.feature.mfcc(y=sig, sr=sr, n_mfcc=40)
    X.append(mfcc_)
    X = np.array(X)
    X = X.reshape(X.shape[0], X.shape[1], X.shape[2], 1)
    # Generate predictions (probabilities -- the output of the last layer)
    # on new data using `predict`
    
    predictions = model.predict(X)
    
    classificacao = np.argmax(predictions, axis = 1)
    print("O som escutado é de um: " + "->" + str(classificacao))
    if classificacao != 3:
     info = [[id,data_atual.strftime("%Y-%m-%d"),data_atual.strftime("%H:%M:%S.%f"),classes[classificacao[0]] ]]
    
     df_existent=pd.read_csv(diames)
     # Criando um DataFrame
     df = pd.DataFrame(info, columns=['id','data','hora','classicacao'])
     #df = pd.concat([df_existent,df], ignore_index=True)
     df.to_csv(diames, index=False, header=False, mode = 'a')
    
     wv.write(f"./assets/{nome_pasta}/{data}_{classes[classificacao[0]]}.wav", recording, freq, sampwidth=2)
     id=id+1
     # Visualizando o DataFrame
     print(df)
    

    
 
    
    
    
    
    
    


Vírgula inicial removida com sucesso.

done...
1/1 [==============================] - 0s 375ms/step
O som escutado é de um: ->[2]
   id        data             hora    classicacao
0   0  2023-09-25  23:03:04.140521  batendo_porta

done...
1/1 [==============================] - 0s 31ms/step
O som escutado é de um: ->[2]
   id        data             hora    classicacao
0   1  2023-09-25  23:03:18.953287  batendo_porta

done...
1/1 [==============================] - 0s 31ms/step
O som escutado é de um: ->[1]
   id        data             hora classicacao
0   2  2023-09-25  23:03:24.203343    cachorro

done...
1/1 [==============================] - 0s 36ms/step
O som escutado é de um: ->[2]
   id        data             hora    classicacao
0   3  2023-09-25  23:03:29.426176  batendo_porta

done...
1/1 [==============================] - 0s 31ms/step
O som escutado é de um: ->[2]
   id        data             hora    classicacao
0   4  2023-09-25  23:03:35.190768  batendo_porta

done...
1/1

In [ ]:
print("./temp/recording"+ str(datetime.now().second) +".wav")

In [ ]:
pip install sounddevice

In [ ]:
pip install wavio

In [ ]:
    recording = sd.rec(int(duration * freq), samplerate=freq, channels=2)
    sd.wait()
    wv.write("./temp/recording1.wav", recording, freq, sampwidth=2)
    print("\ndone...")